## 위/경도 기반 거리 계산 ##

- ex) 도착지 포인트 (1) : 목적지 (4)의 거리, 이동 시간 계산 (직선 거리) and folium 시각화 (Html) 

In [1]:
pip install geopy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [13]:
import folium
from geopy.distance import geodesic
import pandas as pd

def calculate_distance(start_coords, end_coords):
    distance = geodesic(start_coords, end_coords).kilometers
    return distance

def calculate_time(distance, speed_kmph):
    time_hours = distance / speed_kmph
    return time_hours

def plot_route_on_map(start_data, destination_data, average_speed_kmph):
    # 지도의 중심을 출발지로 설정
    map_center = start_data[0]

    # Folium 지도 생성
    my_map = folium.Map(location=map_center, zoom_start=7)

    # 출발지에 마커 추가
    folium.Marker(location=start_data[0], popup=f'출발지: {start_data[1]}', icon=folium.Icon(color='green')).add_to(my_map)

    # 결과 데이터를 저장할 리스트
    result_data = []

    
    
    
    # 여러 목적지에 마커 및 이동 경로 추가
    for dest_coords, dest_name in destination_data:
        # 이동 경로를 선으로 표시 (색상: 빨강)
        folium.PolyLine([start_data[0], (dest_coords[0], dest_coords[1])], color='red').add_to(my_map)

        # 거리 및 시간 계산
        distance = calculate_distance(start_data[0], dest_coords)
        time_hours = calculate_time(distance, average_speed_kmph)

        # 도착지에 빨강색 마커 추가 및 팝업에 거리와 시간 표시
        popup_content = f'도착지: {dest_name}\n거리: {distance:.2f} km\n시간: {time_hours:.2f} 시간'
        folium.Marker(location=(dest_coords[0], dest_coords[1]), popup=popup_content, icon=folium.Icon(color='red')).add_to(my_map)

        # 결과 데이터 저장
        result_data.append({
            '출발지': start_data[1],
            '도착지': dest_name,
            '거리(km)': distance,
            '예상시간(시간)': time_hours
        })
        
        
           
        
        
        print(f"{start_data[1]}에서 {dest_name}까지 거리: {distance:.2f} km, 예상 시간: {time_hours:.2f} 시간")
    
    print(' $$$ 결과 안내 $$$')
    
    
    
    # 결과 데이터를 DataFrame으로 변환
    result_df = pd.DataFrame(result_data)

    # 결과 데이터를 CSV 파일로 저장
    result_df.to_csv('result data.csv', index=False)

    # 지도를 HTML 파일로 저장 (현재 디렉토리에 'route_map.html' 파일로 저장됨)
    my_map.save('multi_point_map.html')

    
    
    
    
if __name__ == "__main__":
    # CSV 파일에서 위도 경도 및 점포명 데이터 읽기
    df = pd.read_csv('raw data.csv')             #############데이터 불러오기#############

    # 출발지 좌표 및 점포명 설정 (첫 번째 행의 데이터를 출발지로 사용)
    start_data = ((df.iloc[0]['y'], df.iloc[0]['x']), df.iloc[0]['점포명'])

    # 목적지 좌표 및 점포명 데이터 생성
    destination_data = [((row['y'], row['x']), row['점포명']) for index, row in df.iloc[1:].iterrows()]

    # 차량의 평균 속도를 60km/h로 가정합니다.
    average_speed_kmph = 60

    # Folium을 사용하여 이동 경로를 지도에 표시하고 HTML 파일로 저장, 결과를 CSV 파일로 저장
    plot_route_on_map(start_data, destination_data, average_speed_kmph)


동탄카림점에서 흑석동점까지 거리: 36.76 km, 예상 시간: 0.61 시간
동탄카림점에서 남부터미널점까지 거리: 32.86 km, 예상 시간: 0.55 시간
동탄카림점에서 분당정자점까지 거리: 19.18 km, 예상 시간: 0.32 시간
동탄카림점에서 서판교점까지 거리: 21.32 km, 예상 시간: 0.36 시간
동탄카림점에서 번동점까지 거리: 48.06 km, 예상 시간: 0.80 시간
동탄카림점에서 당동점까지 거리: 22.56 km, 예상 시간: 0.38 시간
 $$$ 결과 안내 $$$
